# Notebook inputs

In [1]:
import numpy as np
import pandas as pd

In [62]:
TRAINING_DATA_FILE = 'train_data.txt'
TEST_DATA_FILE = 'test_data.txt'

TOKEN_SPAM_PROB_FILE = 'token_spam_probs.txt'
TOKEN_HAM_PROB_FILE = 'token_ham_probs.txt'
TOKEN_ALL_PROB_FILE = 'token_all_probs.txt'

TEST_FEATURE_MATRIX = "test_feature.txt"
TEST_TARGET_FILE = "test_target.txt"

VOCAB_SIZE = 2500


# Read and Load Features from .txt into numpy array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE,delimiter=" ",dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE,delimiter=" ",dtype=int)

In [5]:
sparse_train_data

array([[   0,    2,    1,    1],
       [   0,    3,    1,    2],
       [   0,    4,    1,    1],
       ...,
       [5795, 2271,    0,    1],
       [5795, 2345,    0,    1],
       [5795, 2363,    0,    1]])

In [6]:
sparse_test_data

array([[   8,    2,    1,    1],
       [   8,    3,    1,    4],
       [   8,    4,    1,    2],
       ...,
       [5793, 2381,    0,    1],
       [5793, 2398,    0,    3],
       [5793, 2448,    0,    1]])

In [7]:
print("Nr of rows in training data: ", sparse_train_data.shape[0])
print("Nr of rows in test data: ", sparse_test_data.shape[0])
print("Nr of emails training", np.unique(sparse_train_data[:,0]).size)
print("Nr of emails test", np.unique(sparse_test_data[:,0]).size)

Nr of rows in training data:  258378
Nr of rows in test data:  117775
Nr of emails training 4013
Nr of emails test 1724


In [14]:
column_name = ["DOC_ID"]+["CATEGORY"]+list(range(0,VOCAB_SIZE))
len(column_name)

2502

In [16]:
index_names = np.unique(sparse_train_data[:,0])
index_names[110:120]

array([187, 188, 189, 190, 191, 194, 198, 200, 201, 202])

In [10]:
full_train_data = pd.DataFrame(index=index_names,columns=column_name)
full_train_data.fillna(value=0)

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create full matrix from sparse matrix

In [11]:
def make_full_matrix(sparse_matrix,nr_words,doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    """
    nr_words = size of the vocabulary. Total number of tokens
    doc_idx = position of the document id in the sparse matrix
    word_idx = position of the word id in the sparse matrix
    cat_idx = position of the category id in the sparse matrix
    """
    
    column_name = ["DOC_ID"]+["CAT EGORY"]+list(range(0,nr_words))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names,columns=column_name)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr,"DOC_ID"]= doc_nr
        full_matrix.at[doc_nr, "CATEGORY"] = label
        full_matrix.at[doc_nr,word_id] = occurrence
    
    full_matrix.set_index('DOC_ID',inplace=True)
    return full_matrix
        
        
    

In [17]:
full_matrix_train = make_full_matrix(sparse_train_data,VOCAB_SIZE)
full_matrix_train

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

In [29]:
# probability of spam : % of spam messages in training data set
prob_spam = (full_matrix_train[full_matrix_train["CATEGORY"]==1].shape[0])/full_matrix_train.shape[0]
print("probability of spam is",prob_spam)

probability of spam is 0.310989284824321


In [31]:
# Total number of wordsin emails
train_data_features = full_matrix_train.loc[:,full_matrix_train.columns != "CATEGORY"]

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
words_per_email = train_data_features.sum(axis=1)
total_words = words_per_email.sum(axis=0)
total_words

429246

In [42]:
#Calculate total words in spam mails
spam_emails = full_matrix_train[full_matrix_train["CATEGORY"]==1]
spam_emails_features = spam_emails.loc[:,spam_emails.columns != "CATEGORY"]
spam_word_per_email = spam_emails_features.sum(axis=1)
spam_total_words =  spam_word_per_email.sum(axis=0)
spam_total_words

176343

In [43]:
# Calculate total words in ham mails using words_per_email
ham_word_per_email = words_per_email[full_matrix_train["CATEGORY"]==0]
ham_total_words =  ham_word_per_email.sum(axis=0)
ham_total_words

252903

## Summing the tokens

In [50]:
summed_spam_tokens = spam_emails_features.sum(axis=0)+1

In [51]:
ham_emails = full_matrix_train[full_matrix_train["CATEGORY"]==0]
ham_emails_features = ham_emails.loc[:,ham_emails.columns != "CATEGORY"]
summed_ham_tokens = ham_emails_features.sum(axis=0)+1
summed_ham_tokens

0       5484
1       2590
2       2045
3        938
4       1612
        ... 
2495      16
2496      26
2497      19
2498      22
2499      25
Length: 2500, dtype: int64

# P(Token | Spam) - Probability that a token occures given the email is spam

In [54]:
prob_token_spam = summed_spam_tokens / (spam_total_words+VOCAB_SIZE)
prob_token_spam

0       0.012184
1       0.005228
2       0.006805
3       0.011306
4       0.006816
          ...   
2495    0.000106
2496    0.000050
2497    0.000106
2498    0.000056
2499    0.000045
Length: 2500, dtype: float64

# P(Token | Ham) - Probability that a token occures given the email is Ham

In [55]:
prob_token_ham = summed_ham_tokens / (ham_total_words+VOCAB_SIZE)
prob_token_ham

0       0.021472
1       0.010141
2       0.008007
3       0.003673
4       0.006312
          ...   
2495    0.000063
2496    0.000102
2497    0.000074
2498    0.000086
2499    0.000098
Length: 2500, dtype: float64

# P(Token) - Probability that a token occures

In [56]:
prob_token_all = train_data_features.sum(axis=0) / total_words
prob_token_all

0       0.017848
1       0.008207
2       0.007595
3       0.006891
4       0.006591
          ...   
2495    0.000077
2496    0.000077
2497    0.000084
2498    0.000070
2499    0.000072
Length: 2500, dtype: float64

## Save the probabilities into file

In [59]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_token_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_token_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_token_all)

# Prepare test data

In [63]:
full_matrix_test = make_full_matrix(sparse_test_data,VOCAB_SIZE)
full_matrix_test

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,0,2,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,0,5,5,2,2,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5788,0,0,4,0,2,4,3,3,1,4,...,0,0,0,0,0,0,0,0,0,0


In [69]:
test_data_features = full_matrix_test.loc[:, full_matrix_test.columns!="CATEGORY"]
test_target_values = full_matrix_test.loc[:, full_matrix_test.columns=="CATEGORY"]

In [71]:
np.savetxt(TEST_FEATURE_MATRIX,test_data_features)
np.savetxt(TEST_TARGET_FILE,test_target_values)

In [73]:
test_data_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,1,4,2,1,2,4,1,2,...,0,0,0,0,0,0,0,0,0,0
12,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,2,1,1,2,0,0,3,0,4,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,2,1,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,5,5,2,2,1,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,4,0,2,4,3,3,1,4,3,...,0,0,0,0,0,0,0,0,0,0


(1724, 2500)